<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/MissForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Run on Python3.8**

In [ ]:
!pip install missingpy
!pip install scikit-learn==1.1.2

In [ ]:
import sklearn
print(sklearn.__version__)

1.1.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/WideTable.xlsx')
df = df.iloc[:, 1:]
df.head()

,PatientUID,Recent_Creatinine,Recent_eGFR,Recent_Date,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,...,L27:UrineAlbumin/Cr ratio,L28:UrineProtein/Cr ratio,L29:UrineVolume (24hrs),L30:UrineUrea (24hrs),L31:UrineSodium (24hrs),L32:UrineProtein (24hrs),L1:eGFR,L14:WBC,LabLocation,LabCwhen
0,924852,2.63,15.8,2560-08-07 12:32:50.527,3349148,2.754717,0,30,0,30,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-01 08:15:57.943
1,989278,2.03,32.3,2560-08-11 08:22:48.737,3348711,-46.720000,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,80.3,5.2,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2559-08-01 09:14:12.527
2,2103792,1.05,55.9,2560-08-14 14:38:44.150,3369057,11.018868,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,44.7,7.4,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-08 09:42:53.923
3,953611,1.03,51.1,2560-08-15 02:05:01.323,3366405,17.222520,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,33.5,NaN,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2559-08-06 10:35:19.840
4,488150,1.94,32.6,2560-08-15 09:18:43.240,3356167,-79.002653,0,0,0,0,...,NaN,17.0,NaN,NaN,NaN,NaN,114.2,4.1,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-03 08:44:33.377


In [ ]:
data = pd.DataFrame(df)

In [ ]:
import numpy as np

In [ ]:
columns_to_missfr = [col for col in data.columns
                     if (col not in ['Slopes', 'PatientUID', 'PatientVisitUID',
                                     'InPatient', 'OutPatient', 'Emergency',
                                     'Recent_Date', 'VisitLocation', 'StartDTTM',
                                     'EndDttm', 'LabCwhen'])]

data_selected = data[column_to_mice].copy()


In [ ]:
data_selected

,Recent_Creatinine,Recent_eGFR,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,G1M1:ACEI,...,L26:Vitamin D,L27:UrineAlbumin/Cr ratio,L28:UrineProtein/Cr ratio,L29:UrineVolume (24hrs),L30:UrineUrea (24hrs),L31:UrineSodium (24hrs),L32:UrineProtein (24hrs),L1:eGFR,L14:WBC,LabLocation
0,2.63,15.80,0,30,0,30,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.00,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
1,2.03,32.30,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.30,5.2,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย
2,1.05,55.90,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.70,7.4,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
3,1.03,51.10,0,0,0,0,100,0,0,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.50,NaN,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย
4,1.94,32.60,0,0,0,0,0,0,0,0,...,NaN,NaN,17.0,NaN,NaN,NaN,NaN,114.20,4.1,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52615,1.30,55.28,0,90,0,0,0,0,0,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.30,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
52616,1.30,55.28,0,90,0,90,0,0,0,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.10,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
52617,1.30,55.28,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.60,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
52618,1.30,55.28,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.06,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร


In [ ]:
for col in data_selected.columns:
  print(col, "\t", data_selected[col].dtypes, "\t", np.sum(data_selected[col].isna()))

Recent_Creatinine 	 float64 	 0
Recent_eGFR 	 float64 	 0
G1M2:ARB 	 int64 	 0
G3M1:Statin 	 int64 	 0
G5M2:NSAIDs 	 int64 	 0
G1M5:Diuretic 	 int64 	 0
G1M7:Beta blocker 	 int64 	 0
G1M3:Dihydropyridine CCB 	 int64 	 0
G1M6:MRA 	 int64 	 0
G1M1:ACEI 	 int64 	 0
G2M2:Metformin 	 int64 	 0
G2M3:DPP4 inhibitor 	 int64 	 0
G2M1:SU 	 int64 	 0
G3M3:Fibrate 	 int64 	 0
G2M5:SGLT2i 	 int64 	 0
G4M1:Allopurinol 	 int64 	 0
G1M9:Vasodilator 	 int64 	 0
G1M8:Alpha blocker 	 int64 	 0
G2M6:Insulin 	 int64 	 0
G2M4:TZD 	 int64 	 0
G4M2:Febuxostat 	 int64 	 0
G3M2:Ezetimibe 	 int64 	 0
G1M4:Non-dihydropyridine CCB 	 int64 	 0
G5M1:Ketoanalog amino acid 	 int64 	 0
G2M7:GLP1-RA 	 int64 	 0
G4M3:Uricosuric agent 	 int64 	 0
Sex 	 object 	 0
Age 	 int64 	 0
NATNL 	 object 	 0
Systolic BP 	 float64 	 4690
Diastolic BP 	 float64 	 4712
BMI 	 float64 	 15016
BSA 	 float64 	 15034
D1:Hypertension 	 int64 	 0
D2:Diabetes mellitus 	 int64 	 0
D3:Dyslipidemia 	 int64 	 0
D4:SLE 	 int64 	 0
D5:Systemic scler

In [ ]:
import re
original_columns = data_selected.columns.tolist()
data_selected = data_selected.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '-', x))
data_selected.head()

,Recent_Creatinine,Recent_eGFR,G1M2-ARB,G3M1-Statin,G5M2-NSAIDs,G1M5-Diuretic,G1M7-Beta-blocker,G1M3-Dihydropyridine-CCB,G1M6-MRA,G1M1-ACEI,...,L26-Vitamin-D,L27-UrineAlbumin-Cr-ratio,L28-UrineProtein-Cr-ratio,L29-UrineVolume-24hrs-,L30-UrineUrea-24hrs-,L31-UrineSodium-24hrs-,L32-UrineProtein-24hrs-,L1-eGFR,L14-WBC,LabLocation
0,2.63,15.8,0,30,0,30,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
1,2.03,32.3,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.3,5.2,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย
2,1.05,55.9,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.7,7.4,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
3,1.03,51.1,0,0,0,0,100,0,0,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.5,NaN,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย
4,1.94,32.6,0,0,0,0,0,0,0,0,...,NaN,NaN,17.0,NaN,NaN,NaN,NaN,114.2,4.1,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร


In [ ]:
data_selected['Sex'] = data_selected['Sex'].replace({'ชาย': 0, 'หญิง': 1})

In [ ]:
nationality_map = {'ไทย': 0}

caucasian = ['บริติช  (อังกฤษ, สก็อตแลนด์)', 'อเมริกัน', 'เยอรมัน', 'โอมาน', 'ออสเตรเลีย', 'แคนาดา',
             'อิตาลี', 'สวิส', 'ซีเรีย', 'เดนมาร์ก', 'นอร์เวย์', 'ดัตช์', 'สเปน', 'เบลเยียม', 'ไอซ์แลนด์',
             'ฝรั่งเศส', 'เซนต์คิตส์และเนวิส', 'สวีเดน', 'ออสเตรีย', 'ไซปรัส']
asian = ['ลาว','จีน','เวียดนาม','เขมรอพยพ','ไทยลื้อ','ญี่ปุ่น', 'จีน(ไต้หวัน)']
other = ['ไม่ระบุ','อื่นๆ','ไร้สัญชาติ']

for nationality in asian:
    nationality_map[nationality] = 1

for nationality in caucasian:
    nationality_map[nationality] = 2

for nationality in other:
    nationality_map[nationality] = 3

data_selected['NATNL'] = data_selected['NATNL'].replace(nationality_map)

In [ ]:
data_selected['LabLocation'] = np.where(data_selected['LabLocation'] == "ห้องปฏิบัติการจุลทรรศน์วินิจฉัย", 1, 0)
data_selected['LabLocation'].value_counts()

1    26916
0    25704
Name: LabLocation, dtype: int64

In [ ]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

In [ ]:
imputer = MissForest(random_state=0, verbose=1)
X = data_selected.copy()
X_imputed = imputer.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
/usr/l

In [ ]:
data_missfr = pd.DataFrame(X_imputed, columns=X.columns)
data_missfr

In [ ]:
data_missfr = pd.to_excel('Missfr.xlsx')

#**Get data to contunue**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/WideTable.xlsx')
df = df.iloc[:, 1:]
df.head()

,PatientUID,Recent_Creatinine,Recent_eGFR,Recent_Date,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,...,L15:Platelet,L19:Blood sugar,L20:HbA1c,L21:Cholesterol,L22:Triglyceride,L23:LDL Cholesterol,L24:HDL Cholesterol,L28:UrineProtein/Cr ratio,LabLocation,LabCwhen
0,924852,2.63,15.8,2560-08-07 12:32:50.527,3349148,2.754717,0,30,0,30,...,NaN,NaN,7.2,NaN,NaN,NaN,NaN,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-01 08:15:57.943
1,989278,2.03,32.3,2560-08-11 08:22:48.737,3348711,-46.720000,0,0,0,0,...,480.0,NaN,NaN,202.0,NaN,NaN,NaN,NaN,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2559-08-01 09:14:12.527
2,2103792,1.05,55.9,2560-08-14 14:38:44.150,3369057,11.018868,0,0,0,0,...,442.0,NaN,NaN,157.0,NaN,NaN,NaN,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-08 09:42:53.923
3,953611,1.03,51.1,2560-08-15 02:05:01.323,3366405,17.222520,0,0,0,0,...,NaN,133.0,6.9,201.0,177.0,147.0,30.0,NaN,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2559-08-06 10:35:19.840
4,488150,1.94,32.6,2560-08-15 09:18:43.240,3356167,-79.002653,0,0,0,0,...,253.0,97.0,NaN,NaN,NaN,NaN,NaN,17.0,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-03 08:44:33.377


In [3]:
import pandas as pd
data = pd.read_excel("/content/drive/MyDrive/ckdresearch2023/Assign/Assign 6: MissingValue/Python/MissForest.xlsx")
data.head()

,Recent_Creatinine,Recent_eGFR,G1M2-ARB,G3M1-Statin,G5M2-NSAIDs,G1M5-Diuretic,G1M7-Beta-blocker,G1M3-Dihydropyridine-CCB,G1M6-MRA,G1M1-ACEI,...,L14-WBC,L15-Platelet,L19-Blood-sugar,L20-HbA1c,L21-Cholesterol,L22-Triglyceride,L23-LDL-Cholesterol,L24-HDL-Cholesterol,L28-UrineProtein-Cr-ratio,LabLocation
0,2.63,15.8,0,30,0,30,0,0,0,0,...,7.5704,229.33,143.14,7.200,155.76,116.54,89.24,56.09,98.260,0
1,2.03,32.3,0,0,0,0,0,0,0,0,...,5.2000,480.00,105.80,5.776,202.00,147.08,135.26,51.81,55.979,1
2,1.05,55.9,0,0,0,0,0,0,0,0,...,7.4000,442.00,116.27,6.584,157.00,127.07,87.98,58.98,46.358,0
3,1.03,51.1,0,0,0,0,100,0,0,100,...,8.7333,295.71,133.00,6.900,201.00,177.00,147.00,30.00,145.088,1
4,1.94,32.6,0,0,0,0,0,0,0,0,...,4.1000,253.00,97.00,5.547,185.14,155.93,118.27,58.64,17.000,0


In [4]:
import numpy as np
columns_to_keep  = df[['PatientUID', 'PatientVisitUID', 'Slopes','InPatient', 'OutPatient', 'Emergency','Recent_Date', 'VisitLocation', 'StartDTTM','EndDttm', 'LabCwhen']]
result = pd.concat([columns_to_keep, data], axis=1)
result.head()

,PatientUID,PatientVisitUID,Slopes,InPatient,OutPatient,Emergency,Recent_Date,VisitLocation,StartDTTM,EndDttm,...,L14-WBC,L15-Platelet,L19-Blood-sugar,L20-HbA1c,L21-Cholesterol,L22-Triglyceride,L23-LDL-Cholesterol,L24-HDL-Cholesterol,L28-UrineProtein-Cr-ratio,LabLocation
0,924852,3349148,2.754717,0,1,0,2560-08-07 12:32:50.527,อายุรกรรม 8 (OPD)-D07,2559-08-01 07:20:57.183,2559-08-02 05:01:02.420,...,7.5704,229.33,143.14,7.200,155.76,116.54,89.24,56.09,98.260,0
1,989278,3348711,-46.720000,0,1,0,2560-08-11 08:22:48.737,โรคเลือดและต่อมน้ำเหลือง (คลินิก)-D716,2559-08-01 06:20:26.997,2559-08-02 05:01:02.420,...,5.2000,480.00,105.80,5.776,202.00,147.08,135.26,51.81,55.979,1
2,2103792,3369057,11.018868,0,1,0,2560-08-14 14:38:44.150,อายุรกรรม 9 (OPD)-D08,2559-08-08 07:42:48.500,2559-08-09 05:01:01.760,...,7.4000,442.00,116.27,6.584,157.00,127.07,87.98,58.98,46.358,0
3,953611,3366405,17.222520,0,1,0,2560-08-15 02:05:01.323,บูรณาการนอกเวลา (คลินิก)-OD191,2559-08-06 07:57:29.117,2559-08-07 05:01:01.243,...,8.7333,295.71,133.00,6.900,201.00,177.00,147.00,30.00,145.088,1
4,488150,3356167,-79.002653,0,1,0,2560-08-15 09:18:43.240,บูรณาการนอกเวลา (คลินิก)-OD191,2559-08-03 06:47:08.510,2559-08-04 05:01:02.253,...,4.1000,253.00,97.00,5.547,185.14,155.93,118.27,58.64,17.000,0


In [5]:
columns_with_null  = result.isnull().any()
if columns_with_null.any():
    print("There are columns with null values.")
else:
    print("No columns have null values.")

No columns have null values.


In [6]:
result.to_excel('MissForest.xlsx')